In [20]:
import pyterrier as pt
if not pt.started():
  print ("not")  
  pt.init()

In [21]:
import pandas as pd

# transformers
import pytorch_transformers
from pytorch_transformers import *

# read file
from xml.dom import minidom
import re

#import rank model
from sklearn.ensemble import RandomForestRegressor

# custom extractor
import pickle
from my_functions import extractorRoberta
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
my_extractor = extractorRoberta(my_device = device, model_path = '/notebook/cqas/external_pretrained_models/')
print ("loaded extractors")

from pyterrier import text
from pyterrier.text import scorer

import pyterrier as pt
if not pt.started():
  print ("not")  
  pt.init()

def create_featured_dataset(some_df):
    from pyterrier import text
    from pyterrier.text import scorer
    textscorerTf = text.scorer(body_attr="text", wmodel='BM25', sort=False)
    rtr_bm = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='Tf')
    rtr_tf = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='PL2')
    rtr_pl2 = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='DFIC')
    rtr_dfic = textscorerTf.transform(some_df)
    
    rtr_pl2_for_merge = rtr_pl2[['qid', 'docno', 'score']]
    rtr_pl2_for_merge = rtr_pl2_for_merge.rename(columns={"score": "score_pl2"})
    
    rtr_tf_for_merge = rtr_tf[['qid', 'docno', 'score']]
    rtr_tf_for_merge = rtr_tf_for_merge.rename(columns={"score": "score_tf"})
    
    rtr_bm_for_merge = rtr_bm[['qid', 'docno', 'score']]
    rtr_bm_for_merge = rtr_bm_for_merge.rename(columns={"score": "score_bm"})
    
    rtr_dfic_for_merge = rtr_dfic[['qid', 'docno', 'score']]
    rtr_dfic_for_merge = rtr_dfic_for_merge.rename(columns={"score": "score_dfic"})
    
    result = pd.merge(rtr_pl2_for_merge, rtr_tf_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_bm_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_dfic_for_merge, on=["qid", "docno"])
    result = pd.merge(result, some_df, on=["qid", "docno"])
    zipped = [result["score_pl2"], result["score_tf"], result["score_bm"], result["score_dfic"], result['baseline_scores'], result["is_retrieved"], result["ap_score"], result["objs_score"]]
    unzipped_object = zip(*zipped)
    unzipped_list = list(unzipped_object)
    list_of_features = [np.array(elem) for elem in unzipped_list]
    result['features'] = list_of_features
    return result

def extract_objs_asp(model_for_extraction, input_string):
    model_for_extraction.from_string(input_string)
    obj1, obj2, predicates, aspects = model_for_extraction.get_params()
    return (obj1.lower(), obj2.lower(), predicates, aspects)


def count_score1(text, nlu_tuple):
    (obj1, obj2, pred, asp) = nlu_tuple
    r = 1.0
    if (len(obj1) != 0 and len(obj2) != 0):
        if (len(pred) != 0):
            pred = re.sub('[!#?,.:";]', '', pred[0])
            if (obj1 in text and obj2 in text and pred in text):
                r += 1.0
        if (len(asp) != 0):
            asp = re.sub('[!#?,.:";]', '', asp[0])
            if (obj1 in text and obj2 in text and asp in text):
                r += 1.0
        elif (obj1 in text and obj2 in text):
            r = 1.5
        elif (obj1 in text or obj2 in text):
            r = 1.2
    else:
        if (obj1) in text or (obj2) in text:
            r = 1.2
    return r

def count_score_nlu(nlu_tuple):
    if (len(nlu_tuple[0]) == 0):
        return 0.0
    else: return 1.0


def count_score(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and len(obj2) != 0):
        if (obj1 in text):
            r += 1.0
        if (obj2 in text):
            r += 1.0
        for asp in asps:
            if asp in text:
                r += 1.5
        for pred in preds:
            if pred in text:
                r += 1.0
    else:
        if ((obj1) in text and len(obj1)!= 0) or (obj2 in text and len(obj2) != 0):
            r = 1.0
    return r

def count_score_obj(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and obj1 in text):
        r += 1.0
    if (len(obj2) != 0 and obj2 in text):
        r += 1.0
    return r

def count_score_asp_pred(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    o1 = (len(obj1) != 0 and obj1 in text)
    o2 = (len(obj2) != 0 and obj2 in text)
    if (o1 or o2):
        for asp in asps:
            if asp in text:
                r += 0.5
        for pred in preds:
            if pred in text:
                r += 0.5
    return r


def make_scores_obj(query, answers):
    print ("make_scores_obj")
    (obj1, obj2, pred, asp) = extract_objs_asp(extr, query)
    print ("in make scores", obj1, obj2, pred, asp)
    scores_answers = [count_score(cleanhtml(answer), (obj1, obj2, pred, asp)) for answer in answers]
    return scores_answers

def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list

def make_a_search_request(query):
    # return json
    # json will be processed further
    params = {
            "apikey": "0833a307-97d3-462a-99d9-27db400c70da",
            "query": query,
            "index": ["cw12"],
            "size": 10,
            "pretty": True
        }
    response = requests.get(url = "http://www.chatnoir.eu/api/v1/_search", params = params)
    return response

def clean_punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()


/notebook/cqas/external_pretrained_models/roberta.hdf5
/notebook/cqas/external_pretrained_models/vocab_dir
encoder loaded
indexer loaded
model path  /notebook/cqas/external_pretrained_models/roberta.hdf5
model loaded
reader loaded
loaded extractors


In [22]:
!nvidia-smi

Wed Apr 28 20:11:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   29C    P2    59W / 260W |   3599MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 27%   

In [4]:
#!pip install --upgrade fastrank lightgbm 
#!pip install python-terrier

In [23]:
from baseline import read_xml
topics_2020 = read_xml('topics-task-2.xml')

1
[<DOM Element: topics at 0x7fb1f21e7d70>]
50


In [24]:
topics_2020[45]

('46', '\nWhich beverage has more calories per glass: beer or cider?\n')

In [25]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)
import requests
from xml.dom import minidom
import sys
import argparse

#numpy
import numpy as np

Read retireved documents and qrels:

In [26]:
import pickle


with open('/notebook/touche/list_of_un_answ.pkl', 'rb') as f:
    answers_2020 = pickle.load(f)
    
with open('/notebook/touche2021/touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
    qrels_lines = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
qrels = [x.strip().split() for x in qrels_lines] 


Create qrels dict:

In [27]:
qrels_dict = {}
for elem in qrels:
    query, noninf, docno, rank = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, rank))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, rank))

In [28]:
qrels_dict['46'][:3]

[('clueweb12-1715wb-31-22014', '0'),
 ('clueweb12-0106wb-15-00715', '0'),
 ('clueweb12-1313wb-74-02250', '0')]

In [29]:
from allennlp.common.checks import ConfigurationError
from allennlp.common.file_utils import cached_path
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.dataset_readers.dataset_utils import to_bioul
from allennlp.data.fields import TextField, SequenceLabelField, Field, MetadataField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token

from allennlp.modules.token_embedders import PretrainedTransformerMismatchedEmbedder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.seq2seq_encoders import PassThroughEncoder



from allennlp.data.token_indexers import PretrainedTransformerMismatchedIndexer
from allennlp.data.vocabulary import Vocabulary
from allennlp.predictors import SentenceTaggerPredictor


In [30]:
import pickle
import pandas as pd

topics_2020 = read_xml('topics-task-2.xml')

topics_2021 = read_xml('topics-task-2-only-titles-2021.xml')
        
with open('list_of_un_answ.pcl', 'rb') as f:
    answers_2020 = pickle.load(f)
            
with open('list_of_un_answ_2021.pcl', 'rb') as f:
    answers_2021 = pickle.load(f)
    
with open('touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
    qrels_lines = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
qrels = [x.strip().split() for x in qrels_lines] 

qrels_dict = {}
for elem in qrels:
    query, noninf, docno, rank = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, rank))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, rank))
            
info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
info_df_train = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)
qrels_df_train = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)

1
[<DOM Element: topics at 0x7fb1f27c0f50>]
50
1
[<DOM Element: topics at 0x7fb0a8138410>]
50


In [31]:
def add_ranks1(rtr : pd.DataFrame) -> pd.DataFrame:
    """
        Canonical method for adding a rank column which is calculated based on the score attribute
        for each query. Note that the dataframe is NOT sorted by this operation.
        Arguments
            df: dataframe to create rank attribute for
    """
    rtr.drop(columns=["rank"], errors="ignore", inplace=True)
    if len(rtr) == 0:
        rtr["rank"] = pd.Series(index=rtr.index, dtype='int64')
        return rtr
    print (0)
    # -1 assures that first rank will be FIRST_RANK
    rtr["rank"] = rtr.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + 1
    print (1)
    if True:
        rtr.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)
    return rtr

def transform(model, test_DF):
    """
    Predicts the scores for the given topics.

    Args:
        topicsTest(DataFrame): A dataframe with the test topics.
    """
    test_DF = test_DF.copy()

    # check for change in number of features
    found_numf = test_DF.iloc[0].features.shape[0]
    if model.num_f is not None:
        if found_numf != model.num_f:
            raise ValueError("Expected %d features, but found %d features" % (model.num_f, found_numf))
    if hasattr(model.learner, 'feature_importances_'):
        if len(model.learner.feature_importances_) != found_numf:
            raise ValueError("Expected %d features, but found %d features" % (len(model.learner.feature_importances_), found_numf))

    test_DF["score"] = model.learner.predict(np.stack(test_DF["features"].values))
    return add_ranks1(test_DF)

In [32]:
answs.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,features,score,rank
0,100,clueweb12-0800wb-21-10162,3.846794,6.0,3.012786,4.5161,should i learn python or r for data analysis,i am currently a graduate student in predictiv...,1968.1794,1.0,0.0,2.0,"[3.8467939181105413, 6.0, 3.012786302957528, 4...",0.7,1
1,100,clueweb12-0800wb-21-10162,3.846794,6.0,3.012786,4.5161,should i learn python or r for data analysis,i am currently a graduate student in predictiv...,1968.1794,1.0,0.0,2.0,"[3.8467939181105413, 6.0, 3.012786302957528, 4...",0.7,2
2,100,clueweb12-0800wb-21-10162,3.846794,6.0,3.012786,4.5161,should i learn python or r for data analysis,i am currently a graduate student in predictiv...,1968.1794,1.0,0.0,2.0,"[3.8467939181105413, 6.0, 3.012786302957528, 4...",0.7,3
3,100,clueweb12-0800wb-21-10162,3.846794,6.0,3.012786,4.5161,should i learn python or r for data analysis,i am currently a graduate student in predictiv...,1968.1794,1.0,0.0,2.0,"[3.8467939181105413, 6.0, 3.012786302957528, 4...",0.7,4
4,100,clueweb12-0800wb-21-10162,3.846794,6.0,3.012786,4.5161,should i learn python or r for data analysis,i am currently a graduate student in predictiv...,1968.1794,1.0,0.0,2.0,"[3.8467939181105413, 6.0, 3.012786302957528, 4...",0.7,5


In [33]:
import re

for elem in topics_2021:
    qid, query = elem[0], elem[1].strip('\n')
    query = re.sub(r'[^\w\s]','',query)
    query = cleanhtml(query)
    my_extractor.from_string(query)
    structures = my_extractor.get_params()
        
for ind, answer in enumerate(answers_2021[qid]):
    docno = answer[1]
    score = answer[0]
    text = answer[3]

            #nlu_score = count_score(text, structures)
    objs_score = count_score_obj(text, structures)
    ap_score = count_score_asp_pred(text, structures)
    is_retrieved = count_score_nlu(structures)
    df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
    info_df = info_df.append(df_row, ignore_index= True)
            
    #create train df and crels
    
for elem in topics_2020:
    qid, query = elem[0], elem[1].strip('\n')
    query = re.sub(r'[^\w\s]','',query)
    query = cleanhtml(query)
    my_extractor.from_string(query)
    structures = my_extractor.get_params()

    for ind, answer in enumerate(answers_2020[qid]):
        docno = answer[1]
        score = answer[0]
        text = answer[3]

        nlu_score = count_score(text, structures)
        objs_score = count_score_obj(text, structures)
        ap_score = count_score_asp_pred(text, structures)
        is_retrieved = count_score_nlu(structures)
        df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
        info_df_train = info_df_train.append(df_row, ignore_index= True)
                
    for qrel in qrels_dict[qid]:
        docno, label = qrel
        df_row = {"qid":qid, "docno":docno, "label":label}
        qrels_df_train = qrels_df_train.append(df_row, ignore_index= True)
       
    
print ("info_df head (2021) ", len(info_df), info_df.head())
print ("info_df_test head (2020) ", len(info_df_train), info_df_train.head())
print ("qrels_df_train ", len(qrels_df_train), qrels_df_train.head())
    
    
test_ds = create_featured_dataset(info_df)
result = create_featured_dataset(info_df_train)
    
rf = RandomForestRegressor(n_estimators=20)
rf_pipe = pt.ltr.apply_learned_model(rf)
rf_pipe.fit(result, qrels_df_train)
answs = transform(rf_pipe, test_ds)

in extractor get params 0
self prredicates  
extract_objects_predicates tags ['O', 'O', 'B-Predicate', 'O', 'B-Aspect', 'B-Aspect', 'O', 'O', 'B-Object', 'O', 'B-Object']
extract_objects_predicates words ['what', 'is', 'better', 'at', 'reducing', 'fever', 'in', 'children', 'ibuprofen', 'or', 'aspirin']
2
ind, word 2 better
old start, starts 8 set()
string   at reducing fever in children ibuprofen or aspirin
['ibuprofen', 'aspirin']
['better']
['reducing', 'fever']
len(objects) 2
in extractor get params 0
self prredicates  
extract_objects_predicates tags ['O', 'O', 'O', 'O', 'B-Object', 'O']
extract_objects_predicates words ['what', 'are', 'the', 'best', 'rice', 'cookers']
['rice']
[]
[]
len(objects) 1
in extractor get params 0
self prredicates  
extract_objects_predicates tags ['O', 'O', 'O', 'B-Object', 'O', 'B-Object', 'O']
extract_objects_predicates words ['should', 'i', 'buy', 'steel', 'or', 'ceramic', 'knives']
['steel', 'ceramic']
[]
[]
len(objects) 2
in extractor get params 0
s

In [19]:
qrels_df_train

,qid,docno,label
0,1,clueweb12-0001wb-05-12311,0
1,1,clueweb12-1811wb-62-08424,1
2,1,clueweb12-1811wb-62-08423,1
3,1,clueweb12-1217wb-47-14048,0
4,1,clueweb12-1811wb-62-08425,1
...,...,...,...
5344,50,clueweb12-1200tw-39-15054,0
5345,50,clueweb12-1019wb-89-15610,0
5346,50,clueweb12-1000tw-07-02287,0
5347,50,clueweb12-1300tw-70-00891,0


In [14]:
CUDA_VISIBLE_DEVICES

NameError: name 'CUDA_VISIBLE_DEVICES' is not defined

In [15]:
!export CUDA_LAUNCH_BLOCKING=1

In [27]:
import torch
from allennlp import models
from allennlp.models import SimpleTagger

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

vocab= Vocabulary.from_files("/notebook/cqas/external_pretrained_models/vocab_dir")
BERT_MODEL = 'google/electra-base-discriminator'
embedder = PretrainedTransformerMismatchedEmbedder(model_name=BERT_MODEL)
text_field_embedder = BasicTextFieldEmbedder({'tokens': embedder})
seq2seq_encoder = PassThroughEncoder(input_dim=embedder.get_output_dim())
print ("encoder loaded")
indexer = PretrainedTransformerMismatchedIndexer(model_name=BERT_MODEL)
print ("indexer loaded")
model = SimpleTagger(text_field_embedder=text_field_embedder, 
                      vocab=vocab, 
                      encoder=seq2seq_encoder,
                      calculate_span_f1=True,
                    label_encoding='IOB1')


model.load_state_dict(torch.load("/notebook/cqas/external_pretrained_models/roberta.hdf5", map_location='cuda:0'))

encoder loaded
indexer loaded


<All keys matched successfully>

In [23]:
a = torch.load("/notebook/cqas/external_pretrained_models/bertttt.hdf5")

ModuleNotFoundError: No module named 'src'

In [17]:
dicrt = torch.load("/notebook/NLU_last_version/roberta.hdf5")

RuntimeError: CUDA error: out of memory

In [13]:
next(model.parameters()).is_cuda

False

Model for strucutres retrieval

In [11]:
!nvidia-smi


Mon Apr 26 16:50:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 35%   48C    P2    63W / 260W |   3830MiB / 11019MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 41%   

In [162]:
def extract_objs_asp(model_for_extraction, input_string):
    model_for_extraction.from_string(input_string)
    obj1, obj2, predicates, aspects = model_for_extraction.get_params()
    return (obj1.lower(), obj2.lower(), predicates, aspects)


def count_score1(text, nlu_tuple):
    (obj1, obj2, pred, asp) = nlu_tuple
    r = 1.0
    if (len(obj1) != 0 and len(obj2) != 0):
        if (len(pred) != 0):
            pred = re.sub('[!#?,.:";]', '', pred[0])
            if (obj1 in text and obj2 in text and pred in text):
                r += 1.0
        if (len(asp) != 0):
            asp = re.sub('[!#?,.:";]', '', asp[0])
            if (obj1 in text and obj2 in text and asp in text):
                r += 1.0
        elif (obj1 in text and obj2 in text):
            r = 1.5
        elif (obj1 in text or obj2 in text):
            r = 1.2
    else:
        if (obj1) in text or (obj2) in text:
            r = 1.2
    return r

def make_scores_obj(query, answers):
    print ("make_scores_obj")
    (obj1, obj2, pred, asp) = extract_objs_asp(extr, query)
    print ("in make scores", obj1, obj2, pred, asp)
    scores_answers = [count_score(cleanhtml(answer), (obj1, obj2, pred, asp)) for answer in answers]
    return scores_answers

Create proper pandas df documents and qrels

In [ ]:
import pandas as pd
import re

from help_response import create_list_of_unigue_answers, cleanhtml, clean_punct, read_xml

info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"])
info_df_test = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"])
qrels_df = pd.DataFrame(columns=["qid", "docno", "label"])
qrels_df_test = pd.DataFrame(columns=["qid", "docno", "label"])

for elem in topics_2020:
    qid, query = elem[0], elem[1].strip('\n')
    query = re.sub(r'[^\w\s]','',query)
    query = cleanhtml(query)
    my_extractor.from_string(query)
    structures = my_extractor.get_params()
    
    
    
    for ind, answer in enumerate(answers_2020[qid]):
        docno = answer[1]
        score = answer[0]
        text = answer[3]
        
        nlu_score = count_score(text, structures)
        objs_score = count_score_obj(text, structures)
        ap_score = count_score_asp_pred(text, structures)
        is_retrieved = count_score_nlu(structures)
        df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
        if (int(qid) < 40):
            info_df = info_df.append(df_row, ignore_index= True)
        else:
            info_df_test = info_df_test.append(df_row, ignore_index= True)
            
    for qrel in qrels_dict[qid]:
        docno, label = qrel
        df_row = {"qid":qid, "docno":docno, "label":label}
        if (int(qid) < 40):
            qrels_df = qrels_df.append(df_row, ignore_index= True)
        else:
            qrels_df_test = qrels_df_test.append(df_row, ignore_index= True)
            


#textscorerTf = pt.text.scorer(body_attr="text", wmodel="Tf")
#rtr = textscorerTf.transform(

In [164]:
answers_2020['1'][:3]

[(2406.7341,
  'clueweb12-1214wb-88-29751',
  'difference between love and sex - student.com articles',
  'sex may or may not include penetration. differences between love and sex love love is a feeling (emotional). there is no exact &quot;right&quot; definition of love for everybody. love involves feelings of romance and&#x2f;or attraction. sex: sex is an event or act'),
 (2396.6697,
  'clueweb12-1811wb-62-08418',
  'the difference between rape, sex and love',
  'having ’sex’ and a ‘rape’ are two completely different ideas. while the former is enjoying the feeling between two individual species, the later is aggression of the will power of one over another.'),
 (2270.9827,
  'clueweb12-0200wb-79-18105',
  'helpingteens.org: the differences between love &amp; sex. | sex, sexuality',
  'home &gt; articles &gt; sex, sexuality &amp; pregnancy &gt; the differences between love &amp; sex. love and sex are not the same thing. love is an emotion or a feeling. there is no one definition of lov

In [129]:
from help_response import create_list_of_unigue_answers, cleanhtml, clean_punct, read_xml
text = "sex may or may not include penetration. differences between love and sex love love is a feeling (emotional). there is no exact &quot;right&quot; definition of love for everybody. love involves feelings of romance and&#x2f;or attraction. sex: sex is an event or act'"
text = cleanhtml(text)
structures = ('', '', [], [])
count_score(text, structures)

0.0

In [130]:
info_df.head()

,qid,query,docno,text,baseline_scores,is_retrieved,nlu_score
0,1,what is the difference between sex and love,clueweb12-1214wb-88-29751,sex may or may not include penetration. differ...,2406.7341,0.0,0.0
1,1,what is the difference between sex and love,clueweb12-1811wb-62-08418,having ’sex’ and a ‘rape’ are two completely d...,2396.6697,0.0,0.0
2,1,what is the difference between sex and love,clueweb12-0200wb-79-18105,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.9827,0.0,0.0
3,1,what is the difference between sex and love,clueweb12-1311wb-38-04762,home &gt;&gt;&gt; sex education 2.0 &gt;&gt;&g...,2096.9185,0.0,0.0
4,1,what is the difference between sex and love,clueweb12-0200tw-85-01106,things have changed so much and it has been ye...,2010.6464,0.0,0.0


## Retrieve comaparative sructures

In [131]:
def extract_objs_asp(model_for_extraction, input_string):
    model_for_extraction.from_string(input_string)
    obj1, obj2, predicates, aspects = model_for_extraction.get_params()
    model_for_extraction.clear_params()
    return (obj1.lower(), obj2.lower(), predicates, aspects)

def count_score_nlu(nlu_tuple):
    if (len(nlu_tuple[0]) == 0):
        return 0.0
    else: return 1.0


def count_score(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and len(obj2) != 0):
        if (obj1 in text):
            r += 1.0
        if (obj2 in text):
            r += 1.0
        for asp in asps:
            if asp in text:
                r += 1.5
        for pred in preds:
            if pred in text:
                r += 1.0
    else:
        if ((obj1) in text and len(obj1)!= 0) or (obj2 in text and len(obj2) != 0):
            r = 1.0
    return r

def count_score_obj(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and obj1 in text):
        r += 1.0
    if (len(obj2) != 0 and obj2 in text):
        r += 1.0
    return r

def count_score_asp_pred(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    o1 = (len(obj1) != 0 and obj1 in text)
    o2 = (len(obj2) != 0 and obj2 in text)
    if (o1 or o2):
        for asp in asps:
            if asp in text:
                r += 0.5
        for pred in preds:
            if pred in text:
                r += 0.5
    return r

def make_scores_obj(query, answers):
    print ("make_scores_obj")
    (obj1, obj2, pred, asp) = extract_objs_asp(extr, query)
    print ("in make scores", obj1, obj2, pred, asp)
    scores_answers = [count_score(cleanhtml(answer), (obj1, obj2, pred, asp)) for answer in answers]
    return scores_answers


In [132]:
my_extractor.extract_objects_predicates("What is better Google search or Yahoo search")


extract_objects_predicates tags ['O', 'O', 'B-Predicate', 'B-Object', 'B-Aspect', 'O', 'B-Object', 'B-Aspect']
extract_objects_predicates words ['What', 'is', 'better', 'Google', 'search', 'or', 'Yahoo', 'search']
2
ind, word 2 better
old start, starts -1 set()
string   salary is higher basketball or soccer players
['Google', 'Yahoo']
['better']
['search', 'search']
len(objects) 2


In [133]:
from my_functions import extractorRoberta

In [134]:
import torch
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
my_extractor = extractorRoberta(my_device = device, model_path = '/notebook/cqas/external_pretrained_models/')
print ("loaded extractors")

lambd_extract = lambda qry: extract_objs_asp(my_extractor, qry)

/notebook/cqas/external_pretrained_models/roberta.hdf5
/notebook/cqas/external_pretrained_models/vocab_dir
encoder loaded
indexer loaded
model loaded
reader loaded
loaded extractors


In [135]:
!nvidia-smi

Sat Apr 24 23:06:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 66%   65C    P2   261W / 260W |  10408MiB / 11019MiB |     78%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 68%   

In [136]:
input_string = request.get_data().decode('UTF-8')
my_extractor.from_string(input_string)
obj1, obj2, predicates, aspects = my_extractor.get_params()

NameError: name 'request' is not defined

### Simple ranking

In [166]:
from pyterrier import text

In [167]:
from pyterrier.text import scorer

In [168]:
info_df.head(-15)

,qid,query,docno,text,baseline_scores,is_retrieved,ap_score,objs_score
0,1,what is the difference between sex and love,clueweb12-1214wb-88-29751,sex may or may not include penetration. differ...,2406.73410,0.0,0.0,0.0
1,1,what is the difference between sex and love,clueweb12-1811wb-62-08418,having ’sex’ and a ‘rape’ are two completely d...,2396.66970,0.0,0.0,0.0
2,1,what is the difference between sex and love,clueweb12-0200wb-79-18105,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.98270,0.0,0.0,0.0
3,1,what is the difference between sex and love,clueweb12-1311wb-38-04762,home &gt;&gt;&gt; sex education 2.0 &gt;&gt;&g...,2096.91850,0.0,0.0,0.0
4,1,what is the difference between sex and love,clueweb12-0200tw-85-01106,things have changed so much and it has been ye...,2010.64640,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
36778,39,is java code more readable than code written i...,clueweb12-1215wb-42-08258,this is a clever trick. the real complexity oc...,943.83185,1.0,0.0,0.0
36779,39,is java code more readable than code written i...,clueweb12-1215wb-28-18173,this is a clever trick. the real complexity oc...,943.83185,1.0,0.0,0.0
36780,39,is java code more readable than code written i...,clueweb12-1215wb-30-05633,this is a clever trick. the real complexity oc...,943.83185,1.0,0.0,0.0
36781,39,is java code more readable than code written i...,clueweb12-1215wb-30-05635,this is a clever trick. the real complexity oc...,943.83185,1.0,0.0,0.0


In [169]:
textscorerTf = text.scorer(body_attr="text", wmodel='BM25', sort=False)
rtr_bm = textscorerTf.transform(info_df)

In [170]:
rtr_bm.head()

,qid,docno,rank,score,query
0,1,clueweb12-1412wb-36-28142,0,23.759399,what is the difference between sex and love
1,1,clueweb12-0200wb-79-18105,1,23.572710,what is the difference between sex and love
2,1,clueweb12-1214wb-88-29751,2,23.162551,what is the difference between sex and love
3,1,clueweb12-0601wb-52-24897,3,21.569264,what is the difference between sex and love
4,1,clueweb12-0706wb-92-20025,4,21.569264,what is the difference between sex and love


In [172]:
def create_featured_dataset(some_df):
    textscorerTf = text.scorer(body_attr="text", wmodel='BM25', sort=False)
    rtr_bm = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='Tf')
    rtr_tf = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='PL2')
    rtr_pl2 = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='DFIC')
    rtr_dfic = textscorerTf.transform(some_df)
    
    rtr_pl2_for_merge = rtr_pl2[['qid', 'docno', 'score']]
    rtr_pl2_for_merge = rtr_pl2_for_merge.rename(columns={"score": "score_pl2"})
    
    rtr_tf_for_merge = rtr_tf[['qid', 'docno', 'score']]
    rtr_tf_for_merge = rtr_tf_for_merge.rename(columns={"score": "score_tf"})
    
    rtr_bm_for_merge = rtr_bm[['qid', 'docno', 'score']]
    rtr_bm_for_merge = rtr_bm_for_merge.rename(columns={"score": "score_bm"})
    
    rtr_dfic_for_merge = rtr_dfic[['qid', 'docno', 'score']]
    rtr_dfic_for_merge = rtr_dfic_for_merge.rename(columns={"score": "score_dfic"})
    
    result = pd.merge(rtr_pl2_for_merge, rtr_tf_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_bm_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_dfic_for_merge, on=["qid", "docno"])
    result = pd.merge(result, some_df, on=["qid", "docno"])
    zipped = [result["score_pl2"], result["score_tf"], result["score_bm"], result["score_dfic"], result['baseline_scores'], result["is_retrieved"], result["ap_score"], result["objs_score"]]
    unzipped_object = zip(*zipped)
    unzipped_list = list(unzipped_object)
    list_of_features = [np.array(elem) for elem in unzipped_list]
    result['features'] = list_of_features
    return result

test_ds = create_featured_dataset(info_df_test)
result = create_featured_dataset(info_df)

In [173]:
test_ds.keys()

Index(['qid', 'docno', 'score_pl2', 'score_tf', 'score_bm', 'score_dfic',
       'query', 'text', 'baseline_scores', 'is_retrieved', 'ap_score',
       'objs_score', 'features'],
      dtype='object')

In [145]:
assert set(test_ds['qid']) == set(qrels_df_test['qid'])

## Ranking with LTR

In [154]:
#bm25 = pt.BatchRetrieve(index, wmodel="BM25")
#tf = pt.BatchRetrieve(index, wmodel="Tf")
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
#pipeline = (rtr_tf ** rtr_pl2)

In [174]:
def add_ranks1(rtr : pd.DataFrame) -> pd.DataFrame:
    """
        Canonical method for adding a rank column which is calculated based on the score attribute
        for each query. Note that the dataframe is NOT sorted by this operation.
        Arguments
            df: dataframe to create rank attribute for
    """
    rtr.drop(columns=["rank"], errors="ignore", inplace=True)
    if len(rtr) == 0:
        rtr["rank"] = pd.Series(index=rtr.index, dtype='int64')
        return rtr
    print (0)
    # -1 assures that first rank will be FIRST_RANK
    rtr["rank"] = rtr.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + 1
    print (1)
    if True:
        rtr.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)
    return rtr

def transform(model, test_DF):
    """
    Predicts the scores for the given topics.

    Args:
        topicsTest(DataFrame): A dataframe with the test topics.
    """
    test_DF = test_DF.copy()

    # check for change in number of features
    found_numf = test_DF.iloc[0].features.shape[0]
    if model.num_f is not None:
        if found_numf != model.num_f:
            raise ValueError("Expected %d features, but found %d features" % (model.num_f, found_numf))
    if hasattr(model.learner, 'feature_importances_'):
        if len(model.learner.feature_importances_) != found_numf:
            raise ValueError("Expected %d features, but found %d features" % (len(model.learner.feature_importances_), found_numf))

    test_DF["score"] = model.learner.predict(np.stack(test_DF["features"].values))
    return add_ranks1(test_DF)

In [195]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=20)
rf_pipe = pt.ltr.apply_learned_model(rf)
rf_pipe.fit(result, qrels_df)
answs = transform(rf_pipe, test_ds)
#pt.Experiment([result, rf_pipe], result, qrels_df, ["map"], names=["BM25 Baseline", "LTR"])

0
1


In [185]:
def write_qrels(name, rtr):
    qids = rtr['qid']
    Q0s = [0 for elem in qids]
    docs = rtr['docno']
    scores = rtr['score']
    ranks = rtr['rank']
    tags = [name for elem in qids]
    common_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
    with open(name +'.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
    print ("written " + name +'.qrels')

In [193]:
write_qrels("RF", answs)

written RF.qrels


In [148]:
rtr = qrels_df_test
qids = rtr['qid']
Q0s = [0 for elem in qids]
docs = rtr['docno']
ranks = rtr['label']

common_list = list(zip(qids, Q0s, docs, ranks))
with open("standart_test" +'.qrels', 'w') as fp:
    fp.write('\n'.join('%s %s %s %s' % x for x in common_list))
print ("written " + 'standart' +'_test.qrels')

written standart_test.qrels


In [76]:
qrels_df_test[:3]

,qid,docno,label
0,40,clueweb12-1608wb-39-15329,0
1,40,clueweb12-1700tw-30-03487,0
2,40,clueweb12-0811wb-62-19220,0


In [77]:
#write_qrels("standart", qrels_df_test)

In [78]:
cord19.get_topics(variant='train').head()

NameError: name 'cord19' is not defined

## LightGMB

In [52]:
cord19 = pt.datasets.get_dataset('antique')

In [35]:
topics = cord19.get_topics(variant='train')
qrels = cord19.get_qrels(variant='train')

NameError: name 'cord19' is not defined

In [57]:
import os
pt_index_path = './terrier_cord19_blocks'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

In [58]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tf = pt.BatchRetrieve(index, wmodel="Tf")
pl = pt.BatchRetrieve(index, wmodel="PL2")
dfic = pt.BatchRetrieve(index, wmodel="DFIC")

In [34]:
from sklearn.model_selection import train_test_split

train_topics, test_topics = train_test_split(topics, test_size=15, random_state=42)
train_qrels, test_qrels =  train_test_split(qrels, test_size=15, random_state=42)


NameError: name 'topics' is not defined

In [65]:
ltr_feats1 = (bm25 ** tf ** pl ** dfic)

# for reference, lets record the feature names here too
fnames=["BM25", "TF", 'pl', 'dfic']

In [219]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.1,
      gamma=1.0,
      min_child_weight=0.1,
      max_depth=6,
      verbose=2,
      random_state=42)

rf_lgbm = pt.ltr.apply_learned_model(lmart_x, form="ltr")
rf_lgbm.fit(result, qrels_df, topics_and_results_Valid = test_ds, qrelsValid = qrels_df_test)
answs = transform(rf_lgbm, test_ds)

0
1


Feature importance according LightGBM and XGboost

* f0: result["score_pl2"], 5, 1.76

* f1: result["score_tf"], 0, 1.19
* f2: result["score_bm"], 3, 1.51
* f3: result["score_dfic"], 3, 2.3
* f4: result['baseline_scores'], 19, 23.82
* f5: result["is_retrieved"], 0, 0
* f6: result["ap_score"], 3,  1.66
* f7: result["objs_score"]0, 1.51

In [222]:
rf_lgbm.learner.get_booster().get_score(importance_type="gain")

{'f4': 23.82235418611851,
 'f3': 2.3183860100909097,
 'f0': 1.7651199528500001,
 'f1': 1.19102281,
 'f7': 1.51639706,
 'f6': 1.6681483940833335,
 'f2': 0.9739630702}

In [ ]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=12,
    objective="lambdarank",
    metric="ndcg",
    learning_rate= .1,
    importance_type="gain",
    num_iterations=20,
    early_stopping_rounds=5
)

rf_lgbm = pt.ltr.apply_learned_model(lmart_l, form="ltr")
rf_lgbm.fit(result, qrels_df, topics_and_results_Valid = test_ds, qrelsValid = qrels_df_test)
answs = transform(rf_lgbm, test_ds)



In [227]:
rf_lgbm.learner.booster_.feature_importance()

array([ 5,  0,  3,  3, 19,  0,  3,  0], dtype=int32)

In [218]:
write_qrels("RF", answs)

written RF.qrels


## Custom transformers' score

In [229]:
from transformers_custom import make_scores_transformers, run_baseline

In [ ]:
for head_ in range(11):
    run_baseline(head = head_)

## Bert

In [6]:
import onir_pt

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
!pip3 install onir_pt

ERROR: Could not find a version that satisfies the requirement onir_pt (from versions: none)
ERROR: No matching distribution found for onir_pt
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
#!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
#!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

In [8]:
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='text', vocab_config={'train': True})